In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from time 

ImportError: cannot import name 'datetime' from 'time' (unknown location)

In [3]:
taskA = 'taskA'
taskB = 'taskB'
path = '/Users/fsaxena/Documents/UCL/amls2/AMLSII_19-20_SN14002056/AMLSII_19-20_SN14002056'

taskChoice = taskA

A_df = pd.read_csv(path + '/Datasets/' + taskChoice  + '.csv', sep=',', header=None)

In [4]:
message = A_df[2].values
labels = A_df[1].values

In [5]:
A_df = A_df.rename(columns = {0: "id", 1: "label", 2: "message", 3: "nan"})

In [6]:
def split_train_test(dataframe):
        
        """Taking the datasets that have been computed and then convert them into the respective numpy arrays"""
        message = dataframe['message'] 
        label = dataframe['label']
        
        """Split the data into testing and training sets"""
        """As mentioned, validation set is computed during cross-validation, GridSearch tasks"""

        x, x_test, y, y_test = train_test_split(
            message,
            label,
            test_size=0.2,
            shuffle=True,
            random_state=42,
        )
        
        train_data = (x, y)
        test_data = (x_test, y_test)
        
        return train_data, test_data

In [7]:
training_set, testing_set = split_train_test(A_df)

df1 = pd.DataFrame(data=training_set[0])
df2 = pd.DataFrame(data=training_set[1])
A_df_training = pd.concat([df1, df2], axis=1)

A_df_training = A_df_training.reset_index(drop=True)
A_df_training

,message,label
0,"""it's august 8th and still no Frank Ocean albu...",negative
1,@erix0x Now we want Dunkin' Donuts for breakfa...,positive
2,"""Hey guys, Kris Bryant just hit his 20th homer...",positive
3,"""Red-hot comedian Amy Schumer will bring her f...",neutral
4,Why do the White Sox look like they should be ...,neutral
...,...,...
16501,I don't know if my eyes could handle this matc...,negative
16502,"""@ahmed_muz No, don't think he's ready yet. Wi...",neutral
16503,"""Once again Dustin Johnson is leading early in...",negative
16504,July 19th is National Ice Cream Day in the US....,neutral


In [8]:
x_train = A_df_training['message'].tolist()
y_train = A_df_training['label'].tolist()

In [9]:
# STOP WORDS
# REMOVE PUNCTUATION
# TOKENIZE
# EMBED

In [10]:
import nltk
import random
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
import re

In [11]:
all_words = []
documents = []

from nltk.corpus import stopwords
import re

stop_words = list(set(stopwords.words('english')))

#  j is adject, r is adverb, and v is verb
#allowed_word_types = ["J","R","V"]
allowed_word_types = ["J"]

In [12]:
# for i in range(len(x_train)):
#     print(x_train[i])
    

In [13]:
for p in range(len(x_train)):
#     print(p)
    documents.append((x_train[p], y_train[p]))

    cleaned = re.sub(r'[^(a-zA-Z)\s]','', str(x_train[p]))
    
    tokenized = word_tokenize(cleaned)
    stopped = [w for w in tokenized if not w in stop_words]
    pos = nltk.pos_tag(stopped)
    
#     make a list of  all adjectives identified by the allowed word types list above
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

In [14]:
           
# pickling the list documents to save future recalculations 

save_documents = open("Datasets/saved/documents.pickle","wb")
pickle.dump(documents, save_documents)
save_documents.close()

In [15]:
BOW = nltk.FreqDist(all_words)
BOW

FreqDist({'th': 494, 'new': 427, 'good': 425, 'last': 412, 'nd': 368, 'st': 327, 'best': 276, 'happy': 246, 'next': 241, 'great': 205, ...})

In [16]:
word_features = list(BOW.keys())[:5000]
word_features[0], word_features[-1]

('terror', 'anybody')

In [17]:
save_word_features = open("Datasets/saved/word_features.pickle","wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()

In [18]:
# function to create a dictionary of features for each review in the list document.
# The keys are the words in word_features 
# The values of each key are either true or false for wether that feature appears in the review or not
def find_features(document):
    words = word_tokenize(str(document))
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

# Features mapped to each tweet
featuresets = [(find_features(rev), category) for (rev, category) in documents]

# Shuffling the documents 
random.shuffle(featuresets)
print(len(featuresets))

16506


In [19]:
train_set = 0.8 * len(featuresets)
train_set

13204.800000000001

In [20]:
training_set = featuresets[:int(train_set)]

In [21]:
testing_set = featuresets[int(train_set):]
print( 'training_set :', len(training_set), '\ntesting_set :', len(testing_set))

training_set : 13204 
testing_set : 3302


In [31]:
start = time.time()

classifier = nltk.NaiveBayesClassifier.train(training_set)

print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

classifier.show_most_informative_features(15)
end = time.time() - start
print(str(end) + ' seconds')

Classifier accuracy percent: 59.327680193821934
Most Informative Features
                 excited = True           positi : neutra =     35.5 : 1.0
                   worst = True           negati : positi =     35.4 : 1.0
                  attack = True           negati : positi =     31.3 : 1.0
            manslaughter = True           negati : neutra =     27.3 : 1.0
                 amazing = True           positi : neutra =     26.4 : 1.0
                  stupid = True           negati : positi =     22.6 : 1.0
                   idiot = True           negati : neutra =     22.2 : 1.0
                 awesome = True           positi : neutra =     20.6 : 1.0
               destroyed = True           negati : neutra =     18.0 : 1.0
                  killed = True           negati : positi =     17.8 : 1.0
                marriage = True           negati : positi =     17.2 : 1.0
            disappointed = True           negati : positi =     16.8 : 1.0
                  racist =

In [ ]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC

In [35]:
# print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
# classifier.show_most_informative_features(15)

In [26]:
start = time.time()


MNB_clf = SklearnClassifier(MultinomialNB())
MNB_clf.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_clf, testing_set))*100)

end = time.time() - start
print(str(end) + ' seconds')


MNB_classifier accuracy percent: 59.41853422168383
38.380589962005615 seconds


In [27]:
start = time.time()

BNB_clf = SklearnClassifier(BernoulliNB())
BNB_clf.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BNB_clf, testing_set))*100)

end = time.time() - start
print(str(end) + ' seconds')

BernoulliNB_classifier accuracy percent: 59.96365838885524
44.16016602516174 seconds


In [28]:
start = time.time()

LogReg_clf = SklearnClassifier(LogisticRegression())
LogReg_clf.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogReg_clf, testing_set))*100)

end = time.time() - start
print(str(end) + ' seconds')

/Users/fsaxena/Documents/UCL/amls2/AMLSII_19-20_SN14002056/AMLSII_19-20_SN14002056/venv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression_classifier accuracy percent: 61.50817686250757
87.79885601997375 seconds


In [29]:
start = time.time()

SGD_clf = SklearnClassifier(SGDClassifier())
SGD_clf.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGD_clf, testing_set))*100)

end = time.time() - start
print(str(end) + ' seconds')

SGDClassifier_classifier accuracy percent: 60.629921259842526
53.21599316596985 seconds


In [35]:
def create_pickle(c, file_name): 
    save_classifier = open('Datasets/'+file_name, 'wb')
    pickle.dump(c, save_classifier)
    save_classifier.close()

classifiers_dict = {'ONB': [classifier, 'saved_models/ONB_clf.pickle'],
                    'MNB': [MNB_clf, 'saved_models/MNB_clf.pickle'],
                    'BNB': [BNB_clf, 'saved_models/BNB_clf.pickle'],
                    'LogReg': [LogReg_clf, 'saved_models/LogReg_clf.pickle'],
                    'SGD': [SGD_clf, 'saved_models/SGD_clf.pickle']}

for clf, listy in classifiers_dict.items(): 
    create_pickle(listy[0], listy[1])

In [235]:
# LSTM for sequence classification in the IMDB dataset
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
# top_words = 5000
(X_train, y_train), (X_test, y_test) = (x_train, y_train), (x_test, y_test)
# truncate and pad input sequences
# max_review_length = 1500
# X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
# X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length))
# model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 32)          160000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


/Users/fsaxena/Documents/UCL/amls2/AMLSII_19-20_SN14002056/AMLSII_19-20_SN14002056/venv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3


ValueError: could not convert string to float: '"I offer my sympathy to the Frank Gifford family. Thank you for your years with the NY Giants &amp; Monday Night Football. RIP, Mr. Gifford."'